# VNPY Backtest Test

## Import Modules

In [1]:
from vnpy.app.cta_strategy.backtesting import BacktestingEngine, OptimizationSetting
from vnpy.app.cta_strategy.base import BacktestingMode   
from datetime import datetime    
from vnpy.trader.constant import Interval, Exchange


from vnpy.app.portfolio_strategy.backtesting import BacktestingEngine as PairBacktestEngine

import matplotlib
%matplotlib inline

backtesting的 package 為 vnpy回測包，本身會調用: 指定路徑的database、特殊data數據類型儲存物件、pandas、datetime等 回測所需的function

-BacktestingEngine為回測主引擎，包含交易各項參數設定、加入交易策略、加入特定下單類型、載入回測期間、統計結果、優化參數執行
-OptimizationSetting為參數優化的設定Location，設定特定商品、特定數字區間來完成優化

base 的package定義了CTA模塊中用到的一些基礎設置，如引擎類型（回測/實盤），回測模式（K線/Tick），本地停止單的定義以及停止單狀態（等待中/已撤銷/已觸發）。

-BacktestingMode處理回測類型的設定

-constant的package 本身調用 enum(枚舉)，用來設定特定常數，如交易所類型、持倉資訊、時間間隔

## Trading cost

In [3]:
import re


def getCommission(symbol,
                  exchange,
                  cost,
                  multiplier,
                  qty,
                  Real=False,
                  direction=''):
    """計算個別部位的單邊交易成本"""
    try:
        if exchange == 'TFE':
            if re.match('MTX[0-9]*', symbol):  # 小台指期
                return 50 + (1 * 50 + cost * (2 / 100000) * multiplier
                             ) * qty if not Real else 50 + (cost *
                                                            (2 / 100000) *
                                                            multiplier) * qty
            elif re.match('TX[0-9]*',
                          symbol) or re.match('TE[0-9]*', symbol) or re.match(
                              'TF[0-9]*', symbol):  # 大台指期
                return 50 + (1 * 200 + cost * (2 / 100000) * multiplier
                             ) * qty if not Real else 50 + (cost *
                                                            (2 / 100000) *
                                                            multiplier) * qty
            elif re.match(r"TX[O0-9][0-9]*[A-Z][0-9]", symbol):  # 台指選
                return 30 + (getMinimumTickOpt(cost) + cost * 2 /
                             100000) * qty * multiplier if not Real else 30 + (
                                 cost * 2 / 100000) * qty * multiplier
            elif re.match(r"[A-Z][A-Z]F[0-9][0-9]", symbol):  # 個股期貨
                return 50 + (
                    getMinimumTick(cost) + cost *
                    (2 / 100000)) * qty * multiplier if not Real else 50 + (
                        cost * (2 / 100000)) * qty * multiplier
        elif exchange == 'TSE':
            tick = getMinimumTick(cost)
            commission = cost * (0.1425 / 100) * multiplier
            commission = 20 if commission < 20 else commission
            fee = cost * (0.3 / 100) * multiplier
            slide = tick * multiplier
            if direction == 'EXIT' or direction == 0:
                return (commission + fee +
                        slide) * qty if not Real else (commission + fee) * qty
            else:
                return (commission +
                        slide) * qty if not Real else commission * qty
    except Exception as e:
        print(e)


def getMinimumTickOpt(cost):
    if cost < 10:
        return 0.1
    elif cost < 50:
        return 0.5
    elif cost < 500:
        return 1
    elif cost < 1000:
        return 5
    else:
        return 10


def getMinimumTick(cost):
    if cost < 10:
        return 0.01
    elif cost < 50:
        return 0.05
    elif cost < 100:
        return 0.1
    elif cost < 500:
        return 0.5
    elif cost < 1000:
        return 1
    else:
        return 5

計算交易成本，"getCommission" 參數設定，作為 portfolio_strategy.backtesting 中的Class: BacktestingEngine 中的def: set_parameters 裡手續費價格參數: rates=Dict[str, float]

import re: re為正則表達是模組，可以用來查看字符or字串是否匹配，re.match("MTX[0-9]", symbol) 為輸入的symbol是否相同於MTX0~MTX9

qty為交易數量

multiplier為轉換成數，從price轉換成真實value

用try-except為異常處理的function，Exception為常見錯誤基類

getminimum tick為最小跳動單位設定，Opt為期貨，另一個為股票 (Taiwan's market only)

## setup TurtleSignalStrategy

In [6]:
from vnpy.app.cta_strategy import (           #引入Cta_strategy 會用到的模組，從_init_暫存的memory中個別提取
                                               #(避免提取整個package卻沒讀到需要的模組)
    CtaTemplate,
    StopOrder,
    Direction,
    TickData,
    BarData,
    TradeData,
    OrderData,
    BarGenerator,
    ArrayManager,                          
)


class TurtleSignalStrategy(CtaTemplate):        #繼承CtaTemplate模組
    """"""
    author = "Xi Qi Capital"

    entry_window = 20                           #Parameters
    exit_window = 10
    atr_window = 20
    fixed_size = 1

    entry_up = 0
    entry_down = 0
    exit_up = 0
    exit_down = 0
    atr_value = 0

    long_entry = 0
    short_entry = 0
    long_stop = 0
    short_stop = 0

    parameters = ["entry_window", "exit_window", "atr_window", "fixed_size"]        #parameters 可以在執行前從外部給定參數
    variables = ["entry_up", "entry_down", "exit_up", "exit_down", "atr_value"]   

    def __init__(self, cta_engine, strategy_name, vt_symbol, setting):
        """"""
        super().__init__(cta_engine, strategy_name, vt_symbol, setting)            #super 呼叫父類別的參數，
                                                                                      #用於同個class底下的重複繼承
        self.bg = BarGenerator(self.on_bar)                                       #繼承BarGenerator 的 self.on_bar
        self.am = ArrayManager()                                                  # time series container bar data, 
                                                                                     #calculating technical indocator value
    def on_init(self):
        """
        Callback when strategy is inited.
        """
        self.write_log("策略初始化")
        self.load_bar(20)                                                        #寫入紀錄

    def on_start(self):
        """
        Callback when strategy is started.
        """
        self.write_log("策略启动")

    def on_stop(self):
        """
        Callback when strategy is stopped.
        """
        self.write_log("策略停止")

    def on_tick(self, tick: TickData):
        """
        Callback of new tick data update.
        """
        self.bg.update_tick(tick)              

    def on_bar(self, bar: BarData):
        """
        Callback of new bar data update. 
        """
        self.cancel_all()                           #清空所有成交或未成交的報單 (繼承 CTAtemplate).
        
        #update bar into array manager.             #CTAtemplate內主要運算內容:                               
        self.am.update_bar(bar)                     #update_bar(self, bar)-> None  
                                                      #None為type，表update_bar的output                                  
        if not self.am.inited:                      #update bar前會run是否inited = True，沒有則會 return  
            return                                    #(判斷inited=true, count>=size) 
                                                      #count=計算圈數(K棒)  
                                                      #(沒有符合條件update_bar一樣會計算，只是inited為 False，直到data足夠)
                                                      #size= Data array 資料大小
            

        # Only calculates new entry channel when no position holding      
        if not self.pos:                                                    #position是否為0，是的話才進入indicator計算，
                                                                            #self.pos 在 CTAtemplate預設為0
            self.entry_up, self.entry_down = self.am.donchian(
                self.entry_window
            )

        self.exit_up, self.exit_down = self.am.donchian(self.exit_window)

        if not self.pos:
            self.atr_value = self.am.atr(self.atr_window)           #如果有部位 計算atr，要計算的K棒個數為參數

            self.long_entry = 0                                     #部位開倉值預設                                      
            self.short_entry = 0                                    #部位止損值預設
            self.long_stop = 0
            self.short_stop = 0

            self.send_buy_orders(self.entry_up)                     #send order邏輯，購買價格為donchiain上下界，預掛買進跟賣出
            self.send_short_orders(self.entry_down)                 #觸發任意邊後同方向追單
            
        elif self.pos > 0:                                          #加碼邏輯，倉位多方追多單
                                                                    #self.pos 在 BacktestingEngine裡計算，
                                                                     #pos_change= order.volume, self.pos += pos_change
            self.send_buy_orders(self.entry_up)                     #send_oders 參數為上界                                                     
            sell_price = max(self.long_stop, self.exit_down)        #止損平倉價格，(atr)或 通道下緣取最高者  
            self.sell(sell_price, abs(self.pos), True)              #sell的參數，包含價格、數量(position)

        elif self.pos < 0:                                          #pos_change= -order.volume, self += pos_change
            self.send_short_orders(self.entry_down)

            cover_price = min(self.short_stop, self.exit_up)
            self.cover(cover_price, abs(self.pos), True)

        self.put_event()

    def on_trade(self, trade: TradeData):
        """
        Callback of new trade data update.
        """
        if trade.direction == Direction.LONG:                            #根據抓取新的Data assign給進場price，和停損price
            self.long_entry = trade.price
            self.long_stop = self.long_entry - 2 * self.atr_value        #更新停損的atr
        else:
            self.short_entry = trade.price
            self.short_stop = self.short_entry + 2 * self.atr_value

    def on_order(self, order: OrderData):
        """
        Callback of new order data update.
        """
        pass

    def on_stop_order(self, stop_order: StopOrder):
        """
        Callback of stop order update.
        """
        pass

    def send_buy_orders(self, price):                                    #根據Position volume/fixed size 計算加碼的邏輯
        """"""
        t = self.pos / self.fixed_size

        if t < 1:
            self.buy(price, self.fixed_size, True)

        if t < 2:
            self.buy(price + self.atr_value * 0.5, self.fixed_size, True)

        if t < 3:
            self.buy(price + self.atr_value, self.fixed_size, True)

        if t < 4:
            self.buy(price + self.atr_value * 1.5, self.fixed_size, True)        #最多加碼四口

    def send_short_orders(self, price):
        """"""
        t = self.pos / self.fixed_size

        if t > -1:
            self.short(price, self.fixed_size, True)

        if t > -2:
            self.short(price - self.atr_value * 0.5, self.fixed_size, True)

        if t > -3:
            self.short(price - self.atr_value, self.fixed_size, True)

        if t > -4:
            self.short(price - self.atr_value * 1.5, self.fixed_size, True)


In [7]:
# from vnpy.trader.constant import Interval, Exchange
from functools import partial



tickers = '3008.TSE'                   #選擇標的("標的名稱.交易所代號")
symbol, exchange = tickers.split('.')
get_commission = partial(getCommission, symbol=symbol, exchange=exchange)
engine = BacktestingEngine()

engine.set_parameters(                #設定回測參數，engine
    vt_symbol=tickers,
    interval=Interval.DAILY,
    start=datetime(2010, 7, 1),
    end=datetime(2020, 7, 1),
    rate=get_commission,
    slippage=getMinimumTick,
    size=1000,
    pricetick=getMinimumTick,
    capital=100000,
    mode=BacktestingMode.BAR,
    collection_name='TSE stock') 

engine.add_strategy(TurtleSignalStrategy, {})      #{}內可輸入外部給定參數:
                                                   #"entry_window", "exit_window", "atr_window", "fixed_size"
engine.load_data()
engine.run_backtesting()
df = engine.calculate_result()
engine.calculate_statistics()
engine.show_chart()

2020-11-06 11:38:09.167523	开始加载历史数据
2020-11-06 11:38:09.167523	加载进度： [1%]
2020-11-06 11:38:09.167523	加载进度： [2%]
2020-11-06 11:38:09.167523	加载进度： [2%]
2020-11-06 11:38:09.167523	加载进度： [3%]
2020-11-06 11:38:09.167523	加载进度： [4%]
2020-11-06 11:38:09.167523	加载进度： [5%]
2020-11-06 11:38:09.167523	加载进度： [6%]
2020-11-06 11:38:09.167523	加载进度： [7%]
2020-11-06 11:38:09.167523	加载进度： [7%]
2020-11-06 11:38:09.167523	加载进度： [8%]
2020-11-06 11:38:09.167523	加载进度： [9%]
2020-11-06 11:38:09.167523	加载进度： [10%]
2020-11-06 11:38:09.167523	加载进度：# [11%]
2020-11-06 11:38:09.167523	加载进度：# [11%]
2020-11-06 11:38:09.167523	加载进度：# [12%]
2020-11-06 11:38:09.168521	加载进度：# [13%]
2020-11-06 11:38:09.168521	加载进度：# [14%]
2020-11-06 11:38:09.168521	加载进度：# [15%]
2020-11-06 11:38:09.168521	加载进度：# [16%]
2020-11-06 11:38:09.168521	加载进度：# [16%]
2020-11-06 11:38:09.168521	加载进度：# [17%]
2020-11-06 11:38:09.168521	加载进度：# [18%]
2020-11-06 11:38:09.168521	加载进度：# [19%]
2020-11-06 11:38:09.168521	加载进度：# [20%]
2020-11-06 11:38:09.168521	

C:\Users\j4a3m\anaconda3\lib\site-packages\vnpy\app\cta_strategy\backtesting.py:399: RuntimeWarning:

invalid value encountered in log

